# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

## Key Components

1. Loading and spliting csv files.
2. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
3. Retriever setup for querying the processed documents
4. Creating a question and answer over the csv data.

## Method Details

### Document Preprocessing

1. The csv is loaded using langchain Csvloader
2. The data is split into chunks.


### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the most relevant chunks for a given query.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a csv file.

import libries

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
# Install required packages
!pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# # Load environment variables from a .env file
# load_dotenv()

# # Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [3]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


--2025-09-07 17:18:55--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.04s   

2025-09-07 17:18:55 (5.61 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2025-09-07 17:18:55--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

In [4]:
import pandas as pd

file_path = ('data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


load and process csv data

In [5]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [6]:
docs[-1]

Document(metadata={'source': 'data/customers-100.csv', 'row': 99}, page_content='Index: 100\nCustomer Id: 2354a0E336A91A1\nFirst Name: Clarence\nLast Name: Haynes\nCompany: Le, Nash and Cross\nCity: Judymouth\nCountry: Honduras\nPhone 1: (753)813-6941\nPhone 2: 783.639.1472\nEmail: colleen91@faulkner.biz\nSubscription Date: 2020-03-11\nWebsite: http://www.hatfield-saunders.net/')

Initiate faiss vector store and openai embedding

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-small"
)
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))


/tmp/ipython-input-626138691.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [9]:
print(len(embeddings.embed_query(" "))) # this outputs the dimension of the embedding model...
vector_store = FAISS(
    embedding_function=embeddings,
    index=index, # 384 dimension indexing,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

384


Add the splitted csv data to the vector store

In [10]:
docs[0]

Document(metadata={'source': 'data/customers-100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/')

In [11]:
vector_store.add_documents(documents=docs)

['539eadf4-00f9-4a8e-87af-ae01e65a0b04',
 'c522ca76-5858-43f1-a3d8-fecf34a40f0d',
 'e596d208-5d15-4b50-b61e-c716152d949b',
 '2009086b-3251-4f40-91c5-ae762a9d6afb',
 '224e12a6-0069-4382-acac-7fc797543dc6',
 '408d26b3-0c6e-49a9-ac4f-d09862a9a94e',
 '310a353e-7919-4a7b-b8c9-e05c138bb479',
 'ef7ef26d-35a7-45ee-b4db-6c9f3797d640',
 'eaf09be0-7713-4d29-b88d-f5a50160c417',
 '41ea1858-472b-4f07-9e47-dbed335a1fe3',
 '14123d68-b405-4b49-9f78-5e8e2ec7c4cd',
 'de75373d-c02e-406b-8dfe-bd193094fb2e',
 '4e7b36e1-b989-4a7e-8552-4022c6908d83',
 '928ad21d-9ffe-4838-ac82-ca90fb52dec9',
 'cbbcf5cb-b9d7-4813-9c53-cf48dfa87bb8',
 '6ce23303-8473-4dec-9c07-eb613f9c1635',
 '48df4267-4a91-4289-a58a-5b7733767a47',
 'd0a83aec-7e83-4ffb-848e-4eb3a216ac79',
 '5f4d45cf-106e-4619-a698-309903117187',
 '011b515e-c6d9-4952-808d-38a560a5e04c',
 '651a154c-a5cb-4b3e-b542-e8cd57d3eb2f',
 '7cffd839-2004-45fa-b1ad-cdb063d2dca6',
 '9dfbbce7-11cc-4e4f-ad8e-dfcc6a04a718',
 'edc6e3db-649f-4a14-b8cd-83ceceffd3c2',
 '46c64b25-f57b-

Create the retrieval chain

In [12]:
vector_store.index.ntotal # total 100 vectors with 384 dimensions are indexed..

100

In [13]:
!pip install langchain_huggingface

In [14]:
def create_pipeline(model_id="microsoft/Phi-3-mini-4k-instruct"):
  model_id = "microsoft/Phi-3-mini-4k-instruct"
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id)
  pipe = pipeline(
      "question-answering", model=model, tokenizer=tokenizer, max_new_tokens=10
  )
  hf = HuggingFacePipeline(pipeline=pipe)

  return hf


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFacePipeline


# llm = HuggingFacePipeline(pipeline=model)  # if `model` is a HuggingFace pipeline

retriever = vector_store.as_retriever()


# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# The {context} placeholder is where retrieved documents will be inserted dynamically at runtime.
# Internally what it does is the following

# 1. User asks a question(query)
# 2. Based on embeddings and vector db we do vector_store.search(top_k=3, query)
# 3. We get some results basically relevant docs page content and we joing it
# 4. This is then passed to the llm model as a `context`.
# 5. This context then helps llm produce the output
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),

])
# print(model) # phi3 mini model
# llm = HuggingFacePipeline(pipeline=model)  # if `model` is a HuggingFace pipeline
# Create the question-answer chain
llm = create_pipeline(model_id="microsoft/Phi-3-mini-4k-instruct")
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'Phi3ForCausalLM' is not supported for question-answering. Supported models are ['PeftModelForQuestionAnswering', 'AlbertForQuestionAnswering', 'ArceeForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DiffLlamaForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'Exaone4ForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQue

Query the rag bot with a question based on the CSV data

In [16]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


ValueError: System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/

Index: 9
Customer Id: C2dE4dEEc489ae0
First Name: Sheryl
Last Name: Meyers
Company: Browning-Simon
City: Robersonstad
Country: Cyprus
Phone 1: 854-138-4911x5772
Phone 2: +1-448-910-2276x729
Email: mariokhan@ryan-pope.org
Subscription Date: 2020-01-13
Website: https://www.bullock.net/

Index: 49
Customer Id: F4Fc91fEAEad286
First Name: Brady
Last Name: Cohen
Company: Osborne-Erickson
City: North Eileenville
Country: United Arab Emirates
Phone 1: 741.849.0139x524

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)